<a href="https://colab.research.google.com/github/ganeshiimi/AV/blob/master/Markov_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://towardsdatascience.com/marketing-channel-attribution-with-markov-chains-in-python-part-2-the-complete-walkthrough-733c65b23323
import os
from google.colab import drive
import pandas as pd
import numpy as np

In [ ]:

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/My Drive/AV/Markov_chain')
df=pd.read_csv('attribution data.csv',encoding= 'unicode_escape')

In [ ]:
df = df.sort_values(['cookie', 'time'],
                    ascending=[False, True])
df['visit_order'] = df.groupby('cookie').cumcount() + 1

In [ ]:
df.head()

,cookie,time,interaction,conversion,conversion_value,channel,visit_order
586736,ooooohAFofEnonEikhAi3fF9o,2018-07-14T17:17:12Z,impression,0,0.0,Paid Search,1
586734,ooooiBh70D3k3BfAhDFfii9h7,2018-07-03T12:57:25Z,impression,0,0.0,Paid Search,1
586735,ooooiBh70D3k3BfAhDFfii9h7,2018-07-19T08:17:59Z,impression,0,0.0,Online Video,2
586731,ooooEiB0CCoEf9fiiC90Dfhfk,2018-07-06T23:30:38Z,impression,0,0.0,Online Display,1
586732,ooooEiB0CCoEf9fiiC90Dfhfk,2018-07-12T23:50:45Z,impression,0,0.0,Online Display,2


In [ ]:
df_paths = df.groupby('cookie')['channel'].aggregate(lambda x: x.unique().tolist()).reset_index()
df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='cookie')


In [ ]:
df_paths.head()

,cookie,channel,conversion
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display]",0
1,0000nACkD9nFkBBDECD3ki00E,[Paid Search],0
2,0003EfE37E93D0BC03iBhBBhF,[Paid Search],0
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0


In [ ]:
df_paths['path'] = np.where(df_paths['conversion'] == 0,['Start,']+df_paths['channel'].apply(','.join)+[',Null'],['Start,']+df_paths['channel'].apply(','.join)+[',Conversion'])
df_paths['path'] = df_paths['path'].str.split(',')
df_paths = df_paths[['cookie', 'path']]
df_paths.head()

,cookie,path
0,00000FkCnDfDDf0iC97iC703B,"[Start, Instagram, Online Display, Null]"
1,0000nACkD9nFkBBDECD3ki00E,"[Start, Paid Search, Null]"
2,0003EfE37E93D0BC03iBhBBhF,"[Start, Paid Search, Null]"
3,00073CFE3FoFCn70fBhB3kfon,"[Start, Instagram, Null]"
4,00079hhBkDF3k3kDkiFi9EFAD,"[Start, Paid Search, Null]"


In [ ]:
list_of_paths = df_paths['path']
total_conversions = sum(path.count('Conversion') for path in df_paths['path'].tolist())
base_conversion_rate = total_conversions / len(list_of_paths)

In [ ]:
def transition_states(list_of_paths):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    print(list_of_unique_channels)
    transition_states = {x + '>' + y: 0 for x in list_of_unique_channels for y in list_of_unique_channels}
    print(transition_states)
    for possible_state in list_of_unique_channels:
        if possible_state not in ['Conversion', 'Null']:
            for user_path in list_of_paths:
                if possible_state in user_path:
                    indices = [i for i, s in enumerate(user_path) if possible_state in s]
                    for col in indices:
                        transition_states[user_path[col] + '>' + user_path[col + 1]] += 1

    return transition_states


trans_states = transition_states(list_of_paths)

{'Start', 'Paid Search', 'Online Display', 'Conversion', 'Facebook', 'Online Video', 'Instagram', 'Null'}
{'Start>Start': 0, 'Start>Paid Search': 0, 'Start>Online Display': 0, 'Start>Conversion': 0, 'Start>Facebook': 0, 'Start>Online Video': 0, 'Start>Instagram': 0, 'Start>Null': 0, 'Paid Search>Start': 0, 'Paid Search>Paid Search': 0, 'Paid Search>Online Display': 0, 'Paid Search>Conversion': 0, 'Paid Search>Facebook': 0, 'Paid Search>Online Video': 0, 'Paid Search>Instagram': 0, 'Paid Search>Null': 0, 'Online Display>Start': 0, 'Online Display>Paid Search': 0, 'Online Display>Online Display': 0, 'Online Display>Conversion': 0, 'Online Display>Facebook': 0, 'Online Display>Online Video': 0, 'Online Display>Instagram': 0, 'Online Display>Null': 0, 'Conversion>Start': 0, 'Conversion>Paid Search': 0, 'Conversion>Online Display': 0, 'Conversion>Conversion': 0, 'Conversion>Facebook': 0, 'Conversion>Online Video': 0, 'Conversion>Instagram': 0, 'Conversion>Null': 0, 'Facebook>Start': 0, 'Fac

In [ ]:
trans_states

{'Conversion>Conversion': 0,
 'Conversion>Facebook': 0,
 'Conversion>Instagram': 0,
 'Conversion>Null': 0,
 'Conversion>Online Display': 0,
 'Conversion>Online Video': 0,
 'Conversion>Paid Search': 0,
 'Conversion>Start': 0,
 'Facebook>Conversion': 4690,
 'Facebook>Facebook': 0,
 'Facebook>Instagram': 15252,
 'Facebook>Null': 59319,
 'Facebook>Online Display': 2146,
 'Facebook>Online Video': 2254,
 'Facebook>Paid Search': 4464,
 'Facebook>Start': 0,
 'Instagram>Conversion': 2868,
 'Instagram>Facebook': 10822,
 'Instagram>Instagram': 0,
 'Instagram>Null': 31153,
 'Instagram>Online Display': 1164,
 'Instagram>Online Video': 1193,
 'Instagram>Paid Search': 2266,
 'Instagram>Start': 0,
 'Null>Conversion': 0,
 'Null>Facebook': 0,
 'Null>Instagram': 0,
 'Null>Null': 0,
 'Null>Online Display': 0,
 'Null>Online Video': 0,
 'Null>Paid Search': 0,
 'Null>Start': 0,
 'Online Display>Conversion': 2144,
 'Online Display>Facebook': 2300,
 'Online Display>Instagram': 1251,
 'Online Display>Null': 322

In [ ]:
transition_states

<function __main__.transition_states>